In [2]:
'''
yes24 review crawler
20 reviews for each book
for book that has reviews more than 20
'''

'\nyes24 review crawler\n20 reviews for each book\nfor book that has reviews more than 20\n'

In [46]:
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import numpy as np

In [47]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains

In [48]:
#리뷰페이지 한 쪽에서 리뷰 5개 가져오는 함수
def fiveReview():
    reviews = ''
    reviewBlock = driver.find_element_by_id('infoset_reviewContentList')
    if reviewBlock:
        print('reviewBlock found')

    childNum = 3
    while(True):
        if childNum==8:
            break

        eachReview = reviewBlock.find_element_by_css_selector(f'div:nth-child({childNum}) > div.reviewInfoBot.crop > a')
        print('each review found')

        #더보기 누르고
        btnClick = ActionChains(driver).move_to_element(eachReview).click()
        btnClick.perform()
        print('successfully clicked btn')
        time.sleep(3)

        #내용가져오고
        reviewTxt = reviewBlock.find_element_by_css_selector(f'div:nth-child({childNum}) > div.reviewInfoBot.origin > div.review_cont')
        reviewContent = reviewTxt.text
        #print(reviewContent)
        
        reviews = reviews +' '+reviewContent
        #print(corpus)
        #house.append([isbnNum, reviewContent])
    
        #스크롤 조금 내리고
        driver.execute_script("window.scrollTo(0, window.scrollY + 400);")

        childNum = childNum + 1
    
    return reviews

In [49]:
def crawlReview():
    reviewTemp = ''
    #house = list()
    try:
#         #isbn
#         isbnBlock = driver.find_element_by_css_selector('#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(3) > td')
#         if isbnBlock:
#             print('isbn found')
#             isbnNum = isbnBlock.text
#             print(isbnNum)
            
        reviewPageNum = 4
        for i in range(4):
            print('review {} page'.format(i+1))
            reviewTemp = reviewTemp + fiveReview()
            nextReview = driver.find_element_by_css_selector(f'#infoset_reviewContentList > div.review_sort.sortBot > div.review_sortLft > div > a:nth-child({reviewPageNum})')
            if nextReview:
                nextReviewClick = ActionChains(driver).move_to_element(nextReview).click()
                nextReviewClick.perform()
                time.sleep(10)
                print('successfully clicked next page')
                reviewPageNum = reviewPageNum + 1
        
        print("final reviews for one book:", reviewTemp)
#         house.append([isbnNum,star,corpus])
#         return house
        return reviewTemp
    
    except Exception as e:
        print(e)

In [6]:
data = pd.read_csv('/Users/zogak/hanium/book.csv')

df = data['isbn13']
howMany = len(data['isbn13'])
#for i in range(howMany):
#    print(df[i])
df.head(5)

0    9791165343729
1    9788954681179
2    9791167370280
3    9791191744002
4    9791165341909
Name: isbn13, dtype: int64

In [50]:
chromedriver = '/Users/zogak/chromedriver'
driver = webdriver.Chrome(chromedriver)

driver.get('http://www.yes24.com/main/default.aspx')

data = pd.read_csv('/Users/zogak/hanium/book.csv')

df = data['isbn13']
howMany = len(data['isbn13'])

values = list()
star = 0

for i in range(howMany):
    isbn13 = df[i]
    print(isbn13)
    #검색창
    elem = driver.find_element_by_css_selector("#query")
    elem.clear()
    elem.send_keys("{}".format(isbn13))

    #검색창 클릭
    elem = driver.find_element_by_css_selector("#yesSForm > fieldset > span.schBtn > button")
    elem.send_keys(Keys.RETURN)
    time.sleep(1)
    
    #첫번재 책 클릭
    try:
        firstBook = driver.find_element_by_css_selector('#schMid_wrap > div:nth-child(4) > div.goodsList.goodsList_list > table > tbody > tr:nth-child(1) > td.goods_infogrp > div.goods_rating > span.gd_reviewCount > a > em')
        if firstBook:
            print('book found')
            
            #check how many reviews
            reviewCnt = firstBook.text[:-1]
            reviewCnt = int(reviewCnt)
            print('review : ', reviewCnt)
            
            if reviewCnt < 20:
                print('less than 20')
                driver.back()
                continue
            
            else:
                starBlock = driver.find_element_by_css_selector('#schMid_wrap > div:nth-child(4) > div.goodsList.goodsList_list > table > tbody > tr:nth-child(1) > td.goods_infogrp > div.goods_rating > span.gd_rating > em')
                star = starBlock.text
                print(star)
                
                firstBookClick = driver.find_element_by_css_selector('#schMid_wrap > div:nth-child(4) > div.goodsList.goodsList_list > table > tbody > tr:nth-child(1) > td.goods_infogrp > div.goods_rating > span.gd_reviewCount > a')
                firstBookClick.send_keys(Keys.ENTER)
                reviewCorpus = crawlReview()
                print("finished {}".format(isbn13))
                values.append([isbn13, star, reviewCorpus])
                driver.back()
            
    except Exception as e:
        print(e)
    

9791165343729
book found
invalid literal for int() with base 10: ''
9788954681179
book found
review :  3
less than 20
9791167370280
book found
review :  6
less than 20
9791191744002
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#schMid_wrap > div:nth-child(4) > div.goodsList.goodsList_list > table > tbody > tr:nth-child(1) > td.goods_infogrp > div.goods_rating > span.gd_reviewCount > a > em"}
  (Session info: chrome=92.0.4515.107)

9791165341909
book found
review :  42
9.3
review 1 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 2 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked b

9791191056556
book found
review :  13
less than 20
9791191043297
book found
review :  1
less than 20
9791197103292
book found
invalid literal for int() with base 10: ''
9788937473036
book found
review :  6
less than 20
9791190582476
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#schMid_wrap > div:nth-child(4) > div.goodsList.goodsList_list > table > tbody > tr:nth-child(1) > td.goods_infogrp > div.goods_rating > span.gd_reviewCount > a > em"}
  (Session info: chrome=92.0.4515.107)

9788937461033
book found
review :  21
8.7
review 1 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 2 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully c

9788932321530
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#schMid_wrap > div:nth-child(4) > div.goodsList.goodsList_list > table > tbody > tr:nth-child(1) > td.goods_infogrp > div.goods_rating > span.gd_reviewCount > a > em"}
  (Session info: chrome=92.0.4515.107)

9791190582438
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#schMid_wrap > div:nth-child(4) > div.goodsList.goodsList_list > table > tbody > tr:nth-child(1) > td.goods_infogrp > div.goods_rating > span.gd_reviewCount > a > em"}
  (Session info: chrome=92.0.4515.107)

9788934972204
book found
review :  6
less than 20
9788932921037
book found
review :  8
less than 20
9788933871645
book found
review :  5
less than 20
9788954678407
book found
review :  11
less than 20
9788936434267
book found
review :  41
9.3
review 1 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found

9788937460449
book found
review :  29
8.8
review 1 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 2 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 3 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 4 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
each r

9788954680691
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#schMid_wrap > div:nth-child(4) > div.goodsList.goodsList_list > table > tbody > tr:nth-child(1) > td.goods_infogrp > div.goods_rating > span.gd_reviewCount > a > em"}
  (Session info: chrome=92.0.4515.107)

9788954613729
book found
review :  2
less than 20
9791161571188
book found
review :  1
less than 20
9791190157346
book found
invalid literal for int() with base 10: ''
9791166119880
book found
invalid literal for int() with base 10: ''
9791197182204
book found
review :  1
less than 20
9788932038674
book found
review :  1
less than 20
9788937463822
book found
invalid literal for int() with base 10: ''
9791138400855
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#schMid_wrap > div:nth-child(4) > div.goodsList.goodsList_list > table > tbody > tr:nth-child(1) > td.goods_infogrp > div.goods_rating > span.gd_reviewCount > a > em"}
  (Session inf

In [52]:
len(values)

7

In [53]:
#store values in dataframe
df = pd.DataFrame(values, columns=['isbn13','rating','review'])
print(df.head(3))

          isbn13 rating                                             review
0  9791165341909    9.3    깨어있는 시간도 소중하지만 자는 시간에 큰 행복을 느끼는 나에게 꿈 백화점이라는...
1  9788937461033    8.7   태어나서 죽을 때까지 내내 외롭고, 우울하고, 모든 것이 두려웠던 한 사내가 있다...
2  9788936434267    9.3                                               None


In [54]:
#dataframe to csv
df.to_csv('yes24Review_aladin_1.csv', encoding='utf-8-sig', index=False)